# EPL Data Prep

In [ ]:
import pymysql.cursors
import tensorflow as tf
import pandas as pd 
import qgrid
import pw 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras_tqdm import TQDMNotebookCallback
import numpy as np

from tensorflow.keras.models import model_from_json
json_file = open('regressor.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
regressor = model_from_json(loaded_model_json)
regressor.load_weights("regressor.h5")
 
grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100, 'minVisibleRows': 10}

connection = pymysql.connect(host=pw.srv,
                         user=pw.usr,
                         password=pw.pwrd,
                         db=pw.db,
                         charset='utf8mb4',
                         cursorclass=pymysql.cursors.DictCursor)
def cum_sum (df,groupby_lst,field):
    df['temp'] = df.groupby(groupby_lst)[field].cumsum()
    return df.groupby(groupby_lst)['temp'].shift(1).fillna(0)

def rolled_mean (df,groupby_lst,field, n_months, num_shift):
    _tempDf = pd.DataFrame(df.groupby(groupby_lst)[field].rolling(n_months).mean().reset_index())
    return _tempDf.groupby(groupby_lst)[field].shift(1)

def f(x):
    if x == 0:
        return 0
    elif x == 1:
        return 1
    elif x == 3:
        return 2  
    
def h_goals(row):
    if (row['was_home'] == 1):
        return row['goals_for']
    else:
        return 0 
    
def a_goals(row):
    if (row['was_home'] == 0):
        return row['goals_for']
    else:
        return 0 
    
df= pd.read_sql(""" CALL `epl`.`sp_match_agg_avg_curr`(); """, con=connection)
team_lookup = pd.read_sql(""" SELECT code as id, name FROM epl.tbl_team_info_2018_2019; """, con=connection)  

df['sf'] = str(df['fixture']) + df['season']

 
df['y_vector'] = df.apply(lambda row: f(row['result']), axis=1)

df['win_a'] = df['result'].apply(lambda x: 1 if x == 3 else 0)
df['draw_a'] = df['result'].apply(lambda x: 1 if x == 1 else 0)
df['loss_a'] = df['result'].apply(lambda x: 1 if x == 0 else 0)


filedOrder = ['season','team_id']
numRoll = 3

df['sf'] = df['fixture'].apply(str) + df['season']
df['kp_avg'] = rolled_mean(df,filedOrder,'key_passes',numRoll,1)
df['atp_var'] = df.big_chances_created + df.big_chances_missed + df.target_missed
df['atp_avg'] = rolled_mean(df,filedOrder,'atp_var',numRoll,1)
df['cbi_avg'] = rolled_mean(df,filedOrder,'clearances_blocks_interceptions',numRoll,1)
df['bcc_avg'] = rolled_mean(df,filedOrder,'big_chances_created',numRoll,1)
df['bcm_avg'] = rolled_mean(df,filedOrder,'big_chances_missed',numRoll,1)
df['gf_avg'] = rolled_mean(df,filedOrder,'goals_for',numRoll,1)
df['a_avg'] = rolled_mean(df,filedOrder,'assists',numRoll,1)
df['cp_avg'] = rolled_mean(df,filedOrder,'completed_passes',numRoll,1)
df['ap_avg'] = rolled_mean(df,filedOrder,'attempted_passes',numRoll,1)
df['ap_avg_sq'] = df['ap_avg']**2 
df['pa_avg'] = df['cp_avg']/df['ap_avg']
df['elg_avg'] = rolled_mean(df,filedOrder,'errors_leading_to_goal',numRoll,1)
df['f_avg'] = rolled_mean(df,filedOrder,'fouls',numRoll,1)
df['r_avg'] = rolled_mean(df,filedOrder,'recoveries',numRoll,1)
df['s_avg'] = rolled_mean(df,filedOrder,'saves',numRoll,1)
df['bps_avg'] = rolled_mean(df,filedOrder,'bps',numRoll,1)
df['tgm_avg'] = rolled_mean(df,filedOrder,'target_missed',numRoll,1)
df['cs_avg'] = cum_sum(df,filedOrder,'clean_sheets')
df['win'] = cum_sum(df,filedOrder,'win_a')
df['draw'] = cum_sum(df,filedOrder,'draw_a')
df['loss'] = cum_sum(df,filedOrder,'loss_a')
df['points'] = cum_sum(df,filedOrder,'result')
df['goals_for_reg'] = df['goals_for'] 
df['goals_ag_reg'] = df['goals_against'] 
df['goals_for'] = cum_sum(df,filedOrder,'goals_for')  
df['goals_against'] = cum_sum(df,filedOrder,'goals_against')  
df['goaldiff'] = cum_sum(df,filedOrder,'goals_for')  - cum_sum(df,filedOrder,'goals_against')  

df['h_gf'] = df.apply(h_goals, axis=1)
df['h_gf'] = cum_sum(df,filedOrder,'h_gf')


df['a_gf'] = df.apply(a_goals, axis=1)
df['a_gf'] = cum_sum(df,filedOrder,'a_gf')

prev = next(df.iterrows())[1]
win_streak = []

for index, row in df.iterrows():
    if row['match_day'] == 1 and row['result']== 3:
        streak = 1
        win_streak.append(streak)
    elif row['match_day'] == 1 and row['result'] != 3:
        streak =0
        win_streak.append(streak)
    elif prev['result'] != 3 and row['result'] ==2:
        streak = 1
        win_streak.append(streak)
    elif prev['result'] == 3 and row['result'] ==3:
        streak += 1
        win_streak.append(streak)
    else:
        streak = 0
        win_streak.append(streak)
    prev = row
    

scaler = MinMaxScaler()
encoder = LabelEncoder ()   
    
    
df['winstreak'] = win_streak
df['winstreak'] = df.groupby(filedOrder)['winstreak'].shift(1)


xgprep = df.dropna(subset=['cp_avg','ap_avg','ap_avg_sq','pa_avg','kp_avg','atp_avg'])

xg_df = pd.DataFrame(np.float64(regressor.predict(
                     scaler.fit_transform(xgprep[['cp_avg','ap_avg','ap_avg_sq','pa_avg','kp_avg','atp_avg']]))),index=xgprep.index,columns=['xG'])

df = pd.merge(df, xg_df, how='left', left_index=True, right_index=True)

home_df = pd.merge(df.where(df['was_home']== 1).dropna(), df.where(df['was_home']== 0).dropna(), how='inner', on=['sf'])
away_df = pd.merge(df.where(df['was_home']== 0).dropna(), df.where(df['was_home']== 1).dropna(), how='inner', on=['sf'])
all_df = pd.concat([home_df,away_df])

h_prep_df= (home_df[['sf','team_id_x', 'team_id_y', 'season_x','match_day_x','y_vector_x', 'goaldiff_x',
                      'ap_avg_x','xG_x','cp_avg_x','win_x','pa_avg_x','goals_for_x','goaldiff_y','pa_avg_y',
                      'ap_avg_y','cp_avg_y','xG_y']].dropna())

a_prep_df= (away_df[['y_vector_x','winstreak_x','sf','team_id_x','goaldiff_x','cp_avg_x','ap_avg_x','pa_avg_x','bps_avg_x',
                     'a_avg_x','ap_avg_y','cp_avg_y','goaldiff_y','pa_avg_y','win_y','loss_x']].dropna()) 
 

#learn_prep_df['rslt_num'] = encoder.fit_transform(learn_prep_df['result_x'])
X_h = h_prep_df
Y_h = h_prep_df[['y_vector_x','sf']]

X_a = a_prep_df[[ 'sf','team_id_x','winstreak_x','goaldiff_x','cp_avg_x','ap_avg_x','pa_avg_x','bps_avg_x',
                     'a_avg_x','ap_avg_y','cp_avg_y','goaldiff_y','pa_avg_y','win_y','loss_x']]

#X = pd.DataFrame(scaler.fit_transform(X), index=X.index, columns=X.columns) 
X_h_train, X_h_test, Y_h_train, Y_h_test = train_test_split(X_h, Y_h, test_size=0.40,shuffle=True)

X_a_train = a_prep_df[a_prep_df['sf'].isin(X_h_train['sf'])].drop(columns=['y_vector_x','team_id_x','sf']) 
X_a_test = a_prep_df[a_prep_df['sf'].isin(X_h_test['sf'])].drop(columns=['y_vector_x','team_id_x','sf']) 
Y_a_train = a_prep_df[a_prep_df['sf'].isin(Y_h_train['sf'])]['y_vector_x']
Y_a_test =  a_prep_df[a_prep_df['sf'].isin(Y_h_test['sf'])]['y_vector_x']

X_h_train = X_h_train.drop(columns=['sf','team_id_x', 'team_id_y', 'season_x','match_day_x','y_vector_x']) 
X_h_test = X_h_test.drop(columns=['sf','team_id_x', 'team_id_y', 'season_x','match_day_x','y_vector_x'])
Y_h_train = Y_h_train.drop(columns=['sf'])
Y_h_test = Y_h_test.drop(columns=['sf'])


X_a_train = pd.DataFrame(scaler.fit_transform(X_a_train), index=X_a_train.index, columns=X_a_train.columns) 
X_a_test = pd.DataFrame(scaler.fit_transform(X_a_test), index=X_a_test.index, columns=X_a_test.columns) 
X_h_train = pd.DataFrame(scaler.fit_transform(X_h_train), index=X_h_train.index, columns=X_h_train.columns) 
X_h_test = pd.DataFrame(scaler.fit_transform(X_h_test), index=X_h_test.index, columns=X_h_test.columns) 

#For Keras
Y_keras_train = to_categorical(Y_h_train, num_classes=None)
Y_keras_test = to_categorical(Y_h_test, num_classes=None)

qgrid.show_grid(h_prep_df.corr(),grid_options=grid_options,precision=2)

# Visualizing Correlations with Bokeh

In [ ]:
from bokeh.plotting import *
from bokeh.layouts import column, row, widgetbox
from bokeh.models import CustomJS,HoverTool,ColumnDataSource
from bokeh.palettes import d3
from bokeh.transform import factor_cmap, factor_mark
from bokeh.models.widgets import CheckboxGroup
output_notebook()

h_prep_df2 = pd.merge(team_lookup,home_df, how='inner', left_on=['id'], right_on=['team_id_x'])
h_prep_df2 = pd.merge(h_prep_df2,team_lookup, how='inner', left_on=['team_id_y'], right_on=['id'])

tools = "pan,wheel_zoom,box_zoom,reset,save"
                 
def cats(x):
    if x == 0:
        return "Loss"
    elif x == 1:
        return "Draw"
    elif x == 2:
        return "Win"

h_prep_df2['label'] =  h_prep_df2['y_vector_x'].apply(cats)  

hover = HoverTool(tooltips=[
    ("Team", "@name_x"),
    ("Opponent", "@name_y"),
    ("Season", "@season_x"),
    ("Match Day", "@match_day_x"),
    ("(Expected Goals, Expected Goals Opp)", "(@xG_x, @xG_y)"),
    ('Result', '@label'),
])

source = ColumnDataSource(h_prep_df2)
orig_source = ColumnDataSource(h_prep_df2)

checkbox = CheckboxGroup(labels=h_prep_df2.name_y.unique().tolist())

x = 'xG_x'
y = 'xG_y'
 

checkbox.callback = CustomJS(args=dict(labels=checkbox.labels,source=source,orig=orig_source,x=x,y=y), code="""
   
    const d = orig.data;
    var n;
    currentLabels = []
    for (n in cb_obj.active){
             currentLabels.push(labels[cb_obj.active[n]])
    }     
    
    var index = [] 
    
    for (l in currentLabels) {    
        for(indx in d['name_x']){
            if (d['name_x'][indx] == currentLabels[l]) {
                index.push(indx)
            }
        }
 
    }   
        var temp = []        
        for (var col in orig.data){
            for (var i = 0; i < index.length; i++) {
                 temp.push(orig.data[col][index[i]])
                 }
                 source.data[col] = temp
                 console.log(source.data)
                 temp = []
            } 
        
         
        source.change.emit();
   
""")

#team_selection.js_on_change('active',update)

palette = d3['Category20'][len(h_prep_df2.name_x.unique())]
MARKERS = ['x', 'circle', 'triangle']

p = figure(plot_width=850, plot_height=600,tools=tools)
p.add_tools(hover)



p.scatter(x=x, y=y,size=10,
          color=factor_cmap('name_x', palette, h_prep_df2.name_x.unique()),
          marker=factor_mark('label', MARKERS, h_prep_df2.label.unique()),
          legend='label', source=source)
layout = row(p, widgetbox(checkbox))

show(layout)



# Expected Goals Regressor

In [ ]:
X = df.dropna()

X_h = X[['cp_avg','ap_avg','ap_avg_sq','pa_avg','kp_avg','atp_avg']]
Y_h= X[['goals_for_reg']]
    
X_h_train, X_h_test, Y_h_train, Y_h_test = train_test_split(X_h, Y_h, test_size=0.30,shuffle=True)

X_h_train = pd.DataFrame(scaler.fit_transform(X_h_train), index=X_h_train.index, columns=X_h_train.columns) 
X_h_test = pd.DataFrame(scaler.fit_transform(X_h_test), index=X_h_test.index, columns=X_h_test.columns) 

regressor = Sequential()
regressor.add(Dense(units=4, input_dim=6))
regressor.add(Dense(units=2))
regressor.add(Dense(units=1))
regressor.compile(optimizer='adam', loss='mean_squared_error',  metrics=['mae','accuracy'])
regressor.fit(X_h_train, Y_h_train, epochs=1000, batch_size=40, verbose=2)

regressor_json = regressor.to_json()
with open("regressor.json", "w") as json_file:
    json_file.write(regressor_json)
# serialize weights to HDF5
regressor.save_weights("regressor.h5")
print("Saved model to disk")

# Keras NN 

In [ ]:
model = Sequential()
model.add(Dense(20, input_dim=len(X_h_train.columns), activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

model.fit(X_h_train, Y_keras_train, epochs=10000, batch_size=40)


predictions = model.predict_classes(X_h_test)
predictions
prd_df = pd.DataFrame(predictions, index=X_h_test.index, columns=['pred']) 

cm= confusion_matrix(Y_h_test, predictions)
fig, ax = plot_confusion_matrix(conf_mat=cm)
plt.show()


# Next Game Predicitons

In [ ]:
currentSeason = df.where(df.season == max(df.season)).dropna(how='all') 
nextGames = currentSeason.where(currentSeason.match_day == max(currentSeason.match_day)).dropna(how='all') 
ng_tp = pd.merge(nextGames.where(nextGames['was_home']== 1).dropna(how='all') , nextGames.where(df['was_home']== 0).dropna(how='all') , how='inner', on=['sf'])
ng_tp = pd.merge(team_lookup,ng_tp, how='inner', left_on=['id'], right_on=['opponent_team_id_x'])
ng_tp = pd.merge(team_lookup,ng_tp, how='inner', left_on=['id'], right_on=['team_id_x'])
ng_tp_prep = ng_tp[X_h_train.columns] 
ng_tp_prep = pd.DataFrame(scaler.fit_transform(ng_tp_prep), index=ng_tp_prep.index, columns=ng_tp_prep.columns) 

predictions = model.predict_proba(ng_tp_prep)
prd_df = pd.DataFrame(predictions, index=ng_tp_prep.index, columns=['Loss','Draw','Win']) 
prd_df = pd.merge(ng_tp[['name_x','name_y']],prd_df, how='inner', left_index=True, right_index=True)
qgrid.show_grid(prd_df,grid_options=grid_options,precision=2)